# MSCI Momentum Index Methodology Applied NSE Stocks


## Problem Statement
Replicate MSCI USA Momentum strategy for Indian markets using Bhavcopy package (NSE data)

## Methodology

1. Importing libraries
2. Extracting and storing data from NSE
3. Apply MSCI Methodology 

   Step 1: Selecting appropriate universe of securities (NIFTY 500)
   
   Step 2: Calculating Momentum Score
   
   Step 3: Selecting stocks for Index development
   
   Step 4: Assign weights
   
4. Final Index

## Assumptions/ Notes

1. Used Indian Rupee 3 Month NSE MIBOR rate for Local risk-free rates as per Appendix IV
2. Since weights for Parent index- NIFTY500 were unavailable, I have approximated it as Closing price* total qty traded and used this to calculate market capitalisation weights 
3. Most recent data available from Bhavcopy is used

### Importing necessary libraries

In [37]:
from bhavcopy import bhavcopy
import pandas as pd
import numpy as np
import os
import datetime
from scipy.stats import zscore

### Extracting and storing data from bhavcopy

In [4]:
''' data_storage = "C:/Users/0310a/Desktop/ISS Stoxx/bhav_data"
    
# working directory to save files
os.makedirs(data_storage, exist_ok=True)
os.chdir(data_storage)

start_date = datetime.date(2018, 12, 19)  
end_date = datetime.date(2025, 1, 20)

wait_time = [1, 2]

nse = bhavcopy("equities", start_date, end_date, data_storage, wait_time)
nse.get_data()
print("success")
'''

' data_storage = "C:/Users/0310a/Desktop/ISS Stoxx/bhav_data"\n    \n# working directory to save files\nos.makedirs(data_storage, exist_ok=True)\nos.chdir(data_storage)\n\nstart_date = datetime.date(2018, 12, 19)  \nend_date = datetime.date(2025, 1, 20)\n\nwait_time = [1, 2]\n\nnse = bhavcopy("equities", start_date, end_date, data_storage, wait_time)\nnse.get_data()\nprint("success")\n'

## Step 1: Selecting appropriate universe of securities (NIFTY 500)

In [38]:
equities=pd.read_csv("C:/Users/0310a/Desktop/ISS Stoxx/bhav_data/equities.csv")
equities.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,X
0,20MICRONS,EQ,58.40,58.4,53.60,55.65,55.35,58.95,278417,1.560685e+07,2021-12-20,4091,INE144J01027,NaN
1,21STCENMGM,EQ,51.00,51.0,50.95,50.95,50.95,51.95,1508,7.684250e+04,2021-12-20,48,INE253B01015,NaN
2,3IINFOLTD,EQ,100.10,100.1,100.10,100.10,100.10,105.35,336206,3.365422e+07,2021-12-20,3092,INE748C01038,NaN
3,3MINDIA,EQ,24401.00,24515.8,23760.00,23812.95,23760.00,24536.10,5083,1.213690e+08,2021-12-20,2614,INE470A01017,NaN
4,3PLAND,EQ,12.55,14.3,12.50,12.90,12.85,13.70,10123,1.303276e+05,2021-12-20,144,INE105C01023,NaN


In [39]:
#list of nifty 500 companies 
stocks = pd.read_html("https://en.wikipedia.org/wiki/NIFTY_500")[2]

#fixing issue with header
stocks.columns = stocks.iloc[0] 
stocks = stocks.drop(0)  
stocks = stocks.reset_index(drop=True)
stocks.head()

,Sl.No,Company Name,Industry,Symbol,Series,ISIN Code
0,1,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,2,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,3,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,4,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,5,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026


In [40]:
nifty_500_symbols = stocks['Symbol'].tolist()

In [41]:
# Filtering equities dataset for Nifty 500 companies
filtered_equities = equities[equities['SYMBOL'].isin(nifty_500_symbols)]
filtered_equities.drop("X", axis=1, inplace=True)
filtered_equities.head()

C:\Users\0310a\AppData\Local\Temp\ipykernel_26512\3198835748.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_equities.drop("X", axis=1, inplace=True)


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
3,3MINDIA,EQ,24401.00,24515.80,23760.00,23812.95,23760.0,24536.10,5083,1.213690e+08,2021-12-20,2614,INE470A01017
18,AARTIIND,EQ,950.05,951.55,903.15,909.80,908.7,969.95,801206,7.337770e+08,2021-12-20,42595,INE769A01020
22,AAVAS,EQ,2485.00,2485.00,2261.55,2424.35,2430.0,2529.25,216154,5.107557e+08,2021-12-20,34537,INE216P01012
24,ABB,EQ,2215.00,2215.00,2076.60,2121.65,2145.0,2199.75,102709,2.191432e+08,2021-12-20,12582,INE117A01022
25,ABBOTINDIA,EQ,17950.00,18000.00,17681.65,17917.25,17997.0,18104.90,13512,2.412192e+08,2021-12-20,6281,INE358A01014


In [42]:
filtered_equities.to_csv("filtered_equities.csv")

## Step2: Calculating Momentum Score

### 2.1 Compute Price Momentum

In [43]:
latest_date = filtered_equities['TIMESTAMP'].max()
latest_date

'2024-07-05'

In [44]:
from datetime import timedelta

filtered_equities['TIMESTAMP'] = pd.to_datetime(filtered_equities['TIMESTAMP'])
filtered_equities = filtered_equities.sort_values(by=['TIMESTAMP', 'SYMBOL'], ascending=[False, True])

filtered_equities.head()

C:\Users\0310a\AppData\Local\Temp\ipykernel_26512\3765961804.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_equities['TIMESTAMP'] = pd.to_datetime(filtered_equities['TIMESTAMP'])


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
1518024,360ONE,EQ,927.30,967.70,920.20,964.70,965.0,927.30,498527,4.765996e+08,2024-07-05,38057,INE466L01038
1518030,3MINDIA,EQ,39000.00,39449.95,38600.00,38768.10,38629.0,39525.20,8303,3.238221e+08,2024-07-05,3894,INE470A01017
1518208,AARTIIND,EQ,718.40,728.90,716.15,721.90,721.0,718.45,696843,5.041188e+08,2024-07-05,29091,INE769A01020
1518214,AAVAS,EQ,1780.05,1844.85,1764.30,1779.05,1780.0,1794.85,504158,9.056085e+08,2024-07-05,51614,INE216P01012
1518216,ABB,EQ,8715.95,8715.95,8604.45,8679.40,8675.0,8715.80,222477,1.928285e+09,2024-07-05,32863,INE117A01022


In [46]:
#rebalance rates
latest_date = pd.to_datetime(latest_date)
pt1_date = latest_date - pd.DateOffset(months=1)
pt7_date = latest_date - pd.DateOffset(months=7)  
pt13_date = latest_date - pd.DateOffset(months=13)

In [47]:
def get_price(df, date): #finding prices for calc rebalance dates
    price_row = df[df['TIMESTAMP'] == date]
    if not price_row.empty:
        return price_row['CLOSE'].iloc[0]
    else:
        return None  

In [48]:
def calculate_momentum(df, latest_date):
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

    momentum_values = []     #lists to store mom values

    for company, group in df.groupby('SYMBOL'): #grouping by compnay
        pt1_price = get_price(group, pt1_date)
        pt7_price = get_price(group, pt7_date)
        pt13_price = get_price(group, pt13_date)

        # preventing error- so calc momentum iff prices are available
        if pt1_price is not None and pt7_price is not None:
            six_month_momentum = ((pt1_price / pt7_price) - 1)
        else:
            six_month_momentum = None
        
        if pt1_price is not None and pt13_price is not None:
            twelve_month_momentum = ((pt1_price / pt13_price) - 1)
        else:
            twelve_month_momentum = None

        latest_row = group[group['TIMESTAMP'] == latest_date].iloc[0] #keep only latest date
        
        latest_row['6MonthMomentum'] = six_month_momentum
        latest_row['12MonthMomentum'] = twelve_month_momentum
        
        momentum_values.append(latest_row)

    result_df = pd.DataFrame(momentum_values)

    return result_df

In [49]:
result_df = calculate_momentum(filtered_equities, latest_date)

In [50]:
result_df.isna().sum()

SYMBOL              0
SERIES              0
OPEN                0
HIGH                0
LOW                 0
CLOSE               0
LAST                0
PREVCLOSE           0
TOTTRDQTY           0
TOTTRDVAL           0
TIMESTAMP           0
TOTALTRADES         0
ISIN                0
6MonthMomentum      7
12MonthMomentum    24
dtype: int64

In [51]:
result_df = result_df.dropna(subset=['6MonthMomentum', '12MonthMomentum'])
result_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,6MonthMomentum,12MonthMomentum
1518024,360ONE,EQ,927.30,967.70,920.20,964.70,965.0,927.30,498527,4.765996e+08,2024-07-05,38057,INE466L01038,0.265170,0.825050
1518030,3MINDIA,EQ,39000.00,39449.95,38600.00,38768.10,38629.0,39525.20,8303,3.238221e+08,2024-07-05,3894,INE470A01017,0.055921,0.236773
1518208,AARTIIND,EQ,718.40,728.90,716.15,721.90,721.0,718.45,696843,5.041188e+08,2024-07-05,29091,INE769A01020,0.108477,0.217480
1518214,AAVAS,EQ,1780.05,1844.85,1764.30,1779.05,1780.0,1794.85,504158,9.056085e+08,2024-07-05,51614,INE216P01012,0.039840,0.116685
1518216,ABB,EQ,8715.95,8715.95,8604.45,8679.40,8675.0,8715.80,222477,1.928285e+09,2024-07-05,32863,INE117A01022,0.611996,0.937193


In [52]:
print(result_df.shape)

(474, 15)


### NSE MIBOR rate- Local risk-free rate

In [53]:
MIBOR=pd.read_csv("C:/Users/0310a/Desktop/ISS Stoxx/bhav_data/Term MIBOR.csv")
MIBOR.head()

,Date,Tenor,Time,Term MIBOR,Comments
0,13-Jan-25,3 MONTHS,11:45:00 AM,7.34,NaN
1,10-Jan-25,3 MONTHS,11:45:00 AM,7.33,NaN
2,09-Jan-25,3 MONTHS,11:45:00 AM,7.32,NaN
3,08-Jan-25,3 MONTHS,11:45:00 AM,7.31,NaN
4,07-Jan-25,3 MONTHS,11:45:00 AM,7.28,NaN


In [54]:
result_df['TIMESTAMP'] = pd.to_datetime(result_df['TIMESTAMP'])

def get_mibor_rate(mibor_df, target_date):
    # Since data may not always be there for exact date (holidays etc), finding closest date to the target_date
    closest_row = mibor_df.loc[(mibor_df['Date'] - target_date).abs().idxmin()]
    return closest_row['Term MIBOR']

MIBOR['Date'] = pd.to_datetime(MIBOR['Date'])

latest_date = result_df['TIMESTAMP'].max()

# Calculating rebalancing, 6-month and 12-month dates as per step 1 in msci

pt1_date = latest_date - pd.DateOffset(months=1)
pt7_date = latest_date - pd.DateOffset(months=7)  
pt13_date = latest_date - pd.DateOffset(months=13)

mibor6 = (get_mibor_rate(MIBOR, pt7_date))/100
mibor12 = (get_mibor_rate(MIBOR, pt13_date))/100

print(f"Latest Date (TIMESTAMP): {latest_date}")
print(f"Rebalanced Date (1 month before): {pt1_date}")
print(f"Local Risk-Free Rate (6-month MIBOR): {mibor6}")
print(f"Local Risk-Free Rate (12-month MIBOR): {mibor12}")

Latest Date (TIMESTAMP): 2024-07-05 00:00:00
Rebalanced Date (1 month before): 2024-06-05 00:00:00
Local Risk-Free Rate (6-month MIBOR): 0.0724
Local Risk-Free Rate (12-month MIBOR): 0.0713


### Final price momentum calc

In [55]:
result_df['6MonthMomentum'] -= mibor6
result_df['12MonthMomentum'] -= mibor12
result_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,6MonthMomentum,12MonthMomentum
1518024,360ONE,EQ,927.30,967.70,920.20,964.70,965.0,927.30,498527,4.765996e+08,2024-07-05,38057,INE466L01038,0.192770,0.753750
1518030,3MINDIA,EQ,39000.00,39449.95,38600.00,38768.10,38629.0,39525.20,8303,3.238221e+08,2024-07-05,3894,INE470A01017,-0.016479,0.165473
1518208,AARTIIND,EQ,718.40,728.90,716.15,721.90,721.0,718.45,696843,5.041188e+08,2024-07-05,29091,INE769A01020,0.036077,0.146180
1518214,AAVAS,EQ,1780.05,1844.85,1764.30,1779.05,1780.0,1794.85,504158,9.056085e+08,2024-07-05,51614,INE216P01012,-0.032560,0.045385
1518216,ABB,EQ,8715.95,8715.95,8604.45,8679.40,8675.0,8715.80,222477,1.928285e+09,2024-07-05,32863,INE117A01022,0.539596,0.865893


### 2.2 Computing risk-adjusted momentum

Risk adjusted momentum = Price Momentum / sigma i 

where sigma i = Annualized Standard Deviation of weekly local price returns over the 
period of 3 years 

In [56]:
def calculate_risk_adjusted_momentum(df, momentum_column):

    #  weekly returns
    df['WeeklyReturns'] = df['CLOSE'].pct_change(periods=5)
    df['WeeklyReturns'] = df['WeeklyReturns'].fillna(method='ffill') 

    volatility = df['WeeklyReturns'].rolling(window=156).std() * np.sqrt(52)
    df['AnnualizedVolatility'] = volatility.fillna(method='ffill') 
    df[momentum_column] = df[momentum_column].fillna(df[momentum_column].mean()) 
    df['AnnualizedVolatility'] = df['AnnualizedVolatility'].fillna(df['AnnualizedVolatility'].mean())

    #  risk adjusted momentum
    df[f'{momentum_column}_RiskAdjusted'] = df[momentum_column] / df['AnnualizedVolatility']
    df[f'{momentum_column}_RiskAdjusted'] = df[f'{momentum_column}_RiskAdjusted'].fillna(df[f'{momentum_column}_RiskAdjusted'].mean())
    df.drop(columns=['WeeklyReturns'], inplace=True)

    return df

result_df = calculate_risk_adjusted_momentum(result_df, '6MonthMomentum')
result_df = calculate_risk_adjusted_momentum(result_df, '12MonthMomentum')

In [57]:
result_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,6MonthMomentum,12MonthMomentum,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted
1518024,360ONE,EQ,927.30,967.70,920.20,964.70,965.0,927.30,498527,4.765996e+08,2024-07-05,38057,INE466L01038,0.192770,0.753750,146.754183,0.001314,0.005136
1518030,3MINDIA,EQ,39000.00,39449.95,38600.00,38768.10,38629.0,39525.20,8303,3.238221e+08,2024-07-05,3894,INE470A01017,-0.016479,0.165473,146.754183,-0.000112,0.001128
1518208,AARTIIND,EQ,718.40,728.90,716.15,721.90,721.0,718.45,696843,5.041188e+08,2024-07-05,29091,INE769A01020,0.036077,0.146180,146.754183,0.000246,0.000996
1518214,AAVAS,EQ,1780.05,1844.85,1764.30,1779.05,1780.0,1794.85,504158,9.056085e+08,2024-07-05,51614,INE216P01012,-0.032560,0.045385,146.754183,-0.000222,0.000309
1518216,ABB,EQ,8715.95,8715.95,8604.45,8679.40,8675.0,8715.80,222477,1.928285e+09,2024-07-05,32863,INE117A01022,0.539596,0.865893,146.754183,0.003677,0.005900


### 2.3 Combine Momentum Scores

In [69]:
result_df['6MonthMomentumZ'] = zscore(result_df['6MonthMomentum_RiskAdjusted'])
result_df['12MonthMomentumZ'] = zscore(result_df['12MonthMomentum_RiskAdjusted'])
result_df['CombinedMomentumZ'] = (result_df['6MonthMomentumZ'] + result_df['12MonthMomentumZ']) / 2
result_df.tail()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,ISIN,6MonthMomentum,12MonthMomentum,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore
1520748,ZEEL,EQ,152.0,153.20,149.51,151.10,150.7,150.02,15595180,2.360166e+09,...,INE256A01028,-0.491152,-0.344163,146.994962,-0.003341,-0.002341,-1.521228,-0.879834,-1.200531,0.454436
1520754,ZENSARTECH,EQ,758.0,759.50,732.00,737.20,738.5,752.30,718141,5.320947e+08,...,INE520A01027,0.067434,0.430869,147.024428,0.000459,0.002931,-0.091281,-0.154265,-0.122773,0.890652
1520756,ZFCVINDIA,EQ,15901.0,16047.85,15597.55,15728.35,15700.0,15955.20,8105,1.281629e+08,...,INE342J01019,-0.052148,0.446768,147.016183,-0.000355,0.003039,-0.397358,-0.139357,-0.268358,0.788421
1520760,ZOMATO,EQ,207.0,208.40,204.77,207.57,207.8,207.44,29681228,6.132314e+09,...,INE758T01015,0.501904,1.508349,147.044177,0.003413,0.010258,1.020565,0.854172,0.937368,1.937368
1520765,ZYDUSLIFE,EQ,1145.0,1166.85,1137.05,1162.45,1161.0,1137.05,2734011,3.163353e+09,...,INE010B01027,0.581492,0.984473,148.625088,0.003912,0.006624,1.208412,0.354038,0.781225,1.781225


### 2.4 Winsorize Z-Scores and Compute Momentum Score

In [81]:
result_df['CombinedMomentumZ'] = np.clip(result_df['CombinedMomentumZ'], -3, 3)

#final Momentum Score
result_df['MomentumScore'] = np.where(result_df['CombinedMomentumZ'] > 0,
                                               1 + result_df['CombinedMomentumZ'],
                                               1 / (1 - result_df['CombinedMomentumZ']))

## Step 3. Security Selection and Ranking

In [94]:
index_stocks_final = result_df.sort_values(by='MomentumScore', ascending=False)

In [95]:
index_stocks_final.tail()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,ISIN,6MonthMomentum,12MonthMomentum,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore
1518956,GMMPFAUDLR,EQ,1400.00,1439.90,1391.00,1410.55,1411.90,1404.60,151691,2.148419e+08,...,INE541A01023,-0.274831,-0.251983,52.710442,-0.005214,-0.004781,-2.225931,-1.215538,-1.720734,0.367548
1519729,NESTLEIND,EQ,2550.60,2579.60,2532.50,2574.15,2573.85,2545.15,791513,2.028729e+09,...,INE239A01024,-0.970410,-0.956349,186.211459,-0.005211,-0.005136,-2.224940,-1.264439,-1.744689,0.364340
1518578,CANBK,EQ,117.00,117.98,116.52,117.76,117.85,117.27,20103764,2.361311e+09,...,INE476A01022,-0.807298,-0.702069,146.754183,-0.005501,-0.004784,-2.333951,-1.216015,-1.774983,0.360363
1518610,CGCL,EQ,213.00,214.26,211.50,211.93,212.00,212.85,159299,3.386238e+07,...,INE180C01042,-0.787681,-0.769138,146.754183,-0.005367,-0.005241,-2.283649,-1.278913,-1.781281,0.359547
1518297,ALLCARGO,EQ,64.68,64.99,63.32,64.46,64.40,64.17,3531499,2.268461e+08,...,INE418H01029,-0.833394,-0.836229,146.754183,-0.005679,-0.005698,-2.400865,-1.341833,-1.871349,0.348268


In [96]:
index_stocks_final.shape

(474, 22)

### 3.1 Determining how many stocks to select for index based on Apendix 1

Getting weights for Parent (NIFTY 500) Index

In [97]:
#market cap
index_stocks_final['MarketCap'] = index_stocks_final["TOTTRDVAL"]

# total market cap of index
total_market_cap = index_stocks_final['MarketCap'].sum()

# Market Cap Weight for each stock
index_stocks_final['MarketCapWeight'] = index_stocks_final['MarketCap'] / total_market_cap
index_stocks_final.shape

(474, 24)

In [98]:
index_stocks_final = index_stocks_final.sort_values(by="MarketCap", ascending=False)  
top_25_market_cap = index_stocks_final["MarketCap"].iloc[:25].sum()
top_25_coverage = top_25_market_cap / total_market_cap

# Print results
print(f"Total Market Capitalization: {total_market_cap:.2f}")
print(f"MarketCap of Top 25 Securities: {top_25_market_cap:.2f}")
print(f"Percentage Coverage of Top 25 Securities: {top_25_coverage:.2%}")

Total Market Capitalization: 1285814064613.21
MarketCap of Top 25 Securities: 640854261838.67
Percentage Coverage of Top 25 Securities: 49.84%


In [99]:
index_stocks_final.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,12MonthMomentum,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore,MarketCap,MarketCapWeight
1520094,RVNL,EQ,424.95,498.5,424.55,491.05,492.15,418.75,152345942,7.131340e+10,...,1.949649,189.586316,0.005305,0.010284,1.732588,0.857737,1.295163,2.295163,7.131340e+10,0.055462
1519055,HDFCBANK,EQ,1685.00,1685.0,1642.20,1648.10,1649.40,1727.15,41121274,6.812924e+10,...,-0.104085,50.464914,-0.002312,-0.002063,-1.133950,-0.841462,-0.987706,0.503093,6.812924e+10,0.052985
1519580,MAZDOCK,EQ,5629.60,5860.0,5300.05,5685.80,5653.10,5585.50,9623679,5.468021e+10,...,1.766343,113.597367,0.002868,0.015549,0.815349,1.582417,1.198883,2.198883,5.468021e+10,0.042526
1519243,IRFC,EQ,178.56,191.5,178.10,188.23,187.90,177.72,224563674,4.183283e+10,...,4.162977,111.968189,0.009944,0.037180,3.478136,4.559451,3.000000,4.000000,4.183283e+10,0.032534
1518641,COCHINSHIP,EQ,2715.00,2924.0,2650.00,2837.60,2825.05,2679.95,12633513,3.569451e+10,...,1.942745,146.754183,0.002409,0.013238,0.642702,1.264348,0.953525,1.953525,3.569451e+10,0.027760


### 3.2 Appendix 1 algorithm

In [100]:
def find_num_securities(df, total_securities):

    # is NumSec <= 25 in nifty500 index
    if total_securities <= 25:
        return total_securities

    # what is the no. of sec for target 30% parent Mcap coverage??
    df = df.sort_values(by="MarketCap", ascending=False)  # Ensure correct order
    target_mcap = df["MarketCap"].sum() * 0.3
    cumulative_mcap = df["MarketCap"].cumsum()
    num_sec_target = (cumulative_mcap <= target_mcap).sum()

    # is NumSec <= 25 or <= 10% of parent?
    if num_sec_target <= 25:
        return 25
    elif num_sec_target <= 0.1 * total_securities:
        return max(1, round(0.1 * total_securities))

    num_sec_target = round(num_sec_target)

    # is NumSec >= 40% of parennt NumSec
    if num_sec_target >= 0.4 * total_securities:
        # Reduce NumSec till coverage <= 40%
        while num_sec_target > 0 and (num_sec_target / total_securities) > 0.4:
            num_sec_target -= 1

    # is Mcap Coverage < 20%
    mcap_coverage = cumulative_mcap.iloc[num_sec_target - 1] / df["MarketCap"].sum()
    if mcap_coverage < 0.2:
        # keep increasing NumSec iteratively till Mcap Coverage >= 20%
        while num_sec_target < total_securities and mcap_coverage < 0.2:
            num_sec_target += 1
            mcap_coverage = cumulative_mcap.iloc[num_sec_target - 1] / df["MarketCap"].sum()

    return num_sec_target

final_num_securities = find_num_securities(index_stocks_final, total_securities=500)
print("Final Number of Securities:", final_num_securities)

Final Number of Securities: 25


In [101]:
MSCI_momentum_index_india=index_stocks_final.head(final_num_securities) 
MSCI_momentum_index_india.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,12MonthMomentum,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore,MarketCap,MarketCapWeight
1520094,RVNL,EQ,424.95,498.5,424.55,491.05,492.15,418.75,152345942,7.131340e+10,...,1.949649,189.586316,0.005305,0.010284,1.732588,0.857737,1.295163,2.295163,7.131340e+10,0.055462
1519055,HDFCBANK,EQ,1685.00,1685.0,1642.20,1648.10,1649.40,1727.15,41121274,6.812924e+10,...,-0.104085,50.464914,-0.002312,-0.002063,-1.133950,-0.841462,-0.987706,0.503093,6.812924e+10,0.052985
1519580,MAZDOCK,EQ,5629.60,5860.0,5300.05,5685.80,5653.10,5585.50,9623679,5.468021e+10,...,1.766343,113.597367,0.002868,0.015549,0.815349,1.582417,1.198883,2.198883,5.468021e+10,0.042526
1519243,IRFC,EQ,178.56,191.5,178.10,188.23,187.90,177.72,224563674,4.183283e+10,...,4.162977,111.968189,0.009944,0.037180,3.478136,4.559451,3.000000,4.000000,4.183283e+10,0.032534
1518641,COCHINSHIP,EQ,2715.00,2924.0,2650.00,2837.60,2825.05,2679.95,12633513,3.569451e+10,...,1.942745,146.754183,0.002409,0.013238,0.642702,1.264348,0.953525,1.953525,3.569451e+10,0.027760


## Step 4. Weighing Scheme


Momentum Weight = Momentum Score * Market Capitalization Weight in the Parent Index 

The above weights are then normalized to 100%. The final security level inclusion  factor is determined as the ratio of the final security level weight and the security level pro forma market capitalization weight in the relevant Parent Index.

In [102]:
#Momentum Weight = Momentum Score * Market Capitalization Weight in the Parent Index
mom_score=MSCI_momentum_index_india["MomentumScore"]
mktcap_weight=MSCI_momentum_index_india["MarketCapWeight"]
MSCI_momentum_index_india["Momentum Weight"] = mom_score * mktcap_weight
MSCI_momentum_index_india.sort_values(by="Momentum Weight", ascending=False)
MSCI_momentum_index_india.head()

C:\Users\0310a\AppData\Local\Temp\ipykernel_26512\3079452871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MSCI_momentum_index_india["Momentum Weight"] = mom_score * mktcap_weight


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,AnnualizedVolatility,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore,MarketCap,MarketCapWeight,Momentum Weight
1520094,RVNL,EQ,424.95,498.5,424.55,491.05,492.15,418.75,152345942,7.131340e+10,...,189.586316,0.005305,0.010284,1.732588,0.857737,1.295163,2.295163,7.131340e+10,0.055462,0.127294
1519055,HDFCBANK,EQ,1685.00,1685.0,1642.20,1648.10,1649.40,1727.15,41121274,6.812924e+10,...,50.464914,-0.002312,-0.002063,-1.133950,-0.841462,-0.987706,0.503093,6.812924e+10,0.052985,0.026657
1519580,MAZDOCK,EQ,5629.60,5860.0,5300.05,5685.80,5653.10,5585.50,9623679,5.468021e+10,...,113.597367,0.002868,0.015549,0.815349,1.582417,1.198883,2.198883,5.468021e+10,0.042526,0.093509
1519243,IRFC,EQ,178.56,191.5,178.10,188.23,187.90,177.72,224563674,4.183283e+10,...,111.968189,0.009944,0.037180,3.478136,4.559451,3.000000,4.000000,4.183283e+10,0.032534,0.130136
1518641,COCHINSHIP,EQ,2715.00,2924.0,2650.00,2837.60,2825.05,2679.95,12633513,3.569451e+10,...,146.754183,0.002409,0.013238,0.642702,1.264348,0.953525,1.953525,3.569451e+10,0.027760,0.054230


In [103]:
#Normalising weights to 100
MSCI_momentum_index_india=MSCI_momentum_index_india.copy()
total_momentum_weight = MSCI_momentum_index_india["Momentum Weight"].sum()
MSCI_momentum_index_india["NormalizedMomentumWeight"] = (MSCI_momentum_index_india["Momentum Weight"] / total_momentum_weight) * 100

# Final Index

In [108]:
MSCI_momentum_index_india.sort_values(by="NormalizedMomentumWeight", ascending=False)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,...,6MonthMomentum_RiskAdjusted,12MonthMomentum_RiskAdjusted,6MonthMomentumZ,12MonthMomentumZ,CombinedMomentumZ,MomentumScore,MarketCap,MarketCapWeight,Momentum Weight,NormalizedMomentumWeight
1519243,IRFC,EQ,178.56,191.50,178.10,188.23,187.90,177.72,224563674,4.183283e+10,...,0.009944,0.037180,3.478136,4.559451,3.000000,4.000000,4.183283e+10,0.032534,0.130136,14.204433
1520094,RVNL,EQ,424.95,498.50,424.55,491.05,492.15,418.75,152345942,7.131340e+10,...,0.005305,0.010284,1.732588,0.857737,1.295163,2.295163,7.131340e+10,0.055462,0.127294,13.894130
1519580,MAZDOCK,EQ,5629.60,5860.00,5300.05,5685.80,5653.10,5585.50,9623679,5.468021e+10,...,0.002868,0.015549,0.815349,1.582417,1.198883,2.198883,5.468021e+10,0.042526,0.093509,10.206554
1519240,IRCON,EQ,288.00,315.80,286.50,307.75,308.20,279.75,116642523,3.523541e+10,...,0.002695,0.016977,0.750200,1.778916,1.264558,2.264558,3.523541e+10,0.027403,0.062056,6.773443
1519012,GRSE,EQ,2676.90,2833.80,2546.05,2710.65,2700.50,2668.80,8211761,2.235972e+10,...,0.006106,0.022843,2.034019,2.586234,2.310126,3.310126,2.235972e+10,0.017390,0.057562,6.282865
1518641,COCHINSHIP,EQ,2715.00,2924.00,2650.00,2837.60,2825.05,2679.95,12633513,3.569451e+10,...,0.002409,0.013238,0.642702,1.264348,0.953525,1.953525,3.569451e+10,0.027760,0.054230,5.919256
1519136,HUDCO,EQ,323.90,339.45,318.85,328.45,329.05,325.05,49354323,1.634319e+10,...,0.014441,0.024458,5.170436,2.808487,3.000000,4.000000,1.634319e+10,0.012710,0.050842,5.549368
1519037,HAL,EQ,5535.00,5624.95,5482.50,5552.00,5546.00,5515.10,3446662,1.917971e+10,...,0.012631,0.005114,4.489199,0.146218,2.317708,3.317708,1.917971e+10,0.014916,0.049488,5.401657
1518501,BEML,EQ,4694.80,5488.00,4662.80,5059.95,5082.00,4655.50,6680842,3.462032e+10,...,0.003107,0.009326,0.905416,0.725890,0.815653,1.815653,3.462032e+10,0.026925,0.048886,5.335937
1518500,BEL,EQ,317.65,326.80,314.50,324.05,323.80,317.35,63125211,2.042871e+10,...,0.004364,0.008040,1.378308,0.548925,0.963617,1.963617,2.042871e+10,0.015888,0.031197,3.405214


### Validations

In [109]:
MSCI_momentum_index_india.loc[MSCI_momentum_index_india["NormalizedMomentumWeight"].idxmax()]["SYMBOL"]

'IRFC'

In [110]:
sorted_symbols = sorted(MSCI_momentum_index_india["SYMBOL"].tolist())
print(sorted_symbols)

['AXISBANK', 'BDL', 'BEL', 'BEML', 'BHEL', 'COCHINSHIP', 'DATAPATTNS', 'GRSE', 'HAL', 'HDFCBANK', 'HFCL', 'HUDCO', 'ICICIBANK', 'INFY', 'IRCON', 'IRFC', 'MAZDOCK', 'NBCC', 'ONGC', 'RAILTEL', 'RAYMOND', 'RELIANCE', 'RVNL', 'SBIN', 'YESBANK']


In [111]:
np.round(MSCI_momentum_index_india["NormalizedMomentumWeight"].sum(),2)

100.0

In [112]:
MSCI_momentum_index_india.to_csv('MSCI Momentum Index Final.csv', index=False)

In [113]:
#RETURNS
MSCI_momentum_index_india['Return'] = ((MSCI_momentum_index_india['CLOSE'] - MSCI_momentum_index_india['PREVCLOSE']) / 
                                       MSCI_momentum_index_india['PREVCLOSE']) * 100

totreturn1 = MSCI_momentum_index_india['Return'].sum()

#annualized return
return_old = (np.exp(totreturn1 / 365) - 1) * 100
print("Original MSCI index:", np.round(return_old, 4), "%")

Original MSCI index: 32.8901 %
